In [1]:
import os 

In [2]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://ai-stage2.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = '2023-08-01-preview'
os.environ["OPENAI_API_KEY"] = "cdf63ece486a426da741e31ef181950c"

In [3]:
# os.environ["OPENAI_API_TYPE"] = "azure"
# os.environ["OPENAI_API_BASE"] = "https://ai-stage1.openai.azure.com/"
# os.environ["OPENAI_API_VERSION"] = '2023-08-01-preview'
# os.environ["OPENAI_API_KEY"] = "d3e70cf965e3499c868d5fd0448ecf99"

In [4]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain.chains.llm import LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage
from langchain.chat_models import AzureChatOpenAI
import re
from getpass import getpass
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain.tools import BaseTool
from langchain.memory import ConversationBufferWindowMemory
from langchain import SQLDatabase

from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine


In [5]:
llm_chat_model = AzureChatOpenAI(deployment_name="gpt-4-32k", temperature=0)


In [6]:
url = URL(
    user='jjabborov',
    password='Datox7896',
    account='fsloson-ik76136',
    warehouse='COMPUTE_WH',
    database='PROD_USE_CASES',
    schema='ALAMO',
    role = 'ACCOUNTADMIN'
)
engine = create_engine(url)
connection = engine.connect()


In [7]:
# query = "SELECT * FROM alamo_receivable LIMIT 10;"
# df = pd.read_sql(query, connection)

In [8]:
class CustomSQLDatabase(SQLDatabase):
    def run(
        self,
        command: str,
        fetch = "all",
    ) -> str:
        """Execute a SQL command and return a string representing the results.

        If the statement returns rows, a string of the results is returned.
        If the statement returns no rows, an empty string is returned.
        """
        result = self._execute(command, fetch)
        # Convert columns values to string to avoid issues with sqlalchemy
        # truncating text
        from collections import defaultdict
        values = defaultdict(list)
        for r in result:
            for key, value in r.items():
                values[key].append(value)
        values_dict = dict(values)
        for x in [list, "list"]:
            if x in values_dict.keys():
                values_dict.pop(x)

        if not values_dict:
            return ""
        else:
            return "DataFrame: " + str(values_dict)

In [9]:
db = CustomSQLDatabase(engine)


In [59]:
sql_helper_prompt_template = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct sql query to run on snowflake, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, ALWAYS limit your query to at most 10 results.
User will interact with you while giving you the filters he/she wants. 
If you have messages history and the last actions you did, always try to link the current SQL query to the last one you used.
If you think there is no connection with the current query and the last one, you may not to take the last sql query into account.

User might ask to pivot a table. So here are the rules for this situation:
    - You need to know the distinct values in the asked column to create the pivot table
    - If user didn't request specifically, use SUM function to aggregate.

User might ask you to append multiple tables or produce output table from them. So here are the rules for this situation:
    - You have to append multiple tables so that output must contain all of the columns in the tables user mentioned. 
    - Since all of the columns in the tables are distinct and do not share common name, you MUST use the UNION function in sql query to merge tables into single output table by appending them.
    - The columns in the tables are different, and you MUST fill in the missing columns with NULL values to make the number of columns equal.
    - If there are common column names among the tables, you MUST make them distinctive by merging table's name and column name to create new distinctive name. For example, "BALANCE_FUND" for "BALANCE" is table name and "FUND" is column name.
    - When producing output table, add extra column named 'Source table' and add table's name, or original table's name \
    if you are adding from the last result, in capital letters for each table you are appending. \
    For example, if you are appending the last result with other table and the last result is from 'country' table, add 'COUNTRY' as source table.
    - If user asks to produce output table from the last result, use the last SQL query information to filter the neccasary fields and create output table. Do NOT create any temporary tables.
    - Before producing any sql query to create output table, you should ask what would be the name for the output table from user.


You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

If the question does not seem related to the database, act like helpful assistant.

Here are 6 critical rules for the interaction you must abide:
<rules>
1. When producing query, DO NOT put quotes around the whole query.
2. If you have messages history and the last actions you did, always try to link the current SQL query to the last one you used.
3. Since you are working with snowflake, you have to get table names without the quotes.
4. If you use alias as temporary name for column, put it under double quotes.
4. If you have to return choices or options, present them one more time at the end of the response in new line in this specific format so I can parse it: \n<list>car,balance,table</list>
5. If user asks you to produce output table, make sure to ask the name for desired output table.
6. If you produced output table, you MUST write output table name at the end of the message: \n<output_name>table_name</output_name>
7. If your Observation (the result of the action) starts with "DataFrame: ", DO NOT RETURN the result of that observation as your final answer, just return 'Here it is'.
8. When you produced  query, make sure to close with semicolon (;) at the end.
</rules>


You have access to the following to these tools below:

{tools}

You MUST use one of the 2 following formats:

1-format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action 
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

2-format:
Question: the input question you must answer
Thought: I need additional information on that question
Final Answer: the question to ask for futher development of the analysis

Message history is here below:
{history}

The last 5 actions you did are here below:
{last_five_actions}

Question: {input}
{agent_scratchpad}
"""

In [60]:
import ast 
import pandas as pd
def getting_df(r):
    output = r["output"]
    if r["intermediate_steps"]:
        last_observation = r["intermediate_steps"][-1][-1]
        if "DataFrame: " in last_observation:
            last_sql_input = r["intermediate_steps"][-1][0].tool_input
            dataframe = pd.read_sql(last_sql_input, connection)
            return dataframe

In [61]:
# import ast 
# import pandas as pd
# def getting_df(r):
#     output = r["output"]
#     if "DataFrame: " in output:
#         data_frame_str = output.split("DataFrame: ")[1]
#         data_frame = ast.literal_eval(data_frame_str)
#         return pd.DataFrame(data_frame)

In [62]:
"""6. If your Observation (the result of the action) starts with "DataFrame: ", RETURN the exacty result of that observation with little information about what user asked as your final answer. \
For example, if observation is "DataFrame: {{column1: ['country1', 'country2']}}", return "Here is the first 10 rows of country table\nDataFrame: {{column1: ['country1', 'country2']}}"."""
"""    - If there are the columns created in previous actions and do not exist in the actual table, you need recreate the query that produced these columns and use it as a subquery in the new query. 
"""
"""If user asks to do anything with the result you gave them by saying "this/that/above result", use the last EXACT query you produced.
"""
"""User will interact with you while giving you the filters he/she wants. 
If you user recalls the filters they asked by, you have to recall which query or queries they are asking. If you need help, ask them to confirm. 
User might ask to produce a table after several steps. Make sure to apply all the necessary filters they asked before."""

"""    - If user asks you to produce output table from "this/that/above result", use the latest EXACT query to produce output table
You can order the results by a relevant column to return the most interesting examples in the database.
"""

"""The lastr query you did is here below:
{last_sql_query}"""

"""If user asks to do anything with the result you gave them by saying "this/that/above result", take into past queries you produced into account.
"""
"""If user asks you to append multiple tables or to produce output table from multiple tables, you have to append multiple tables so that output must contain all of the columns in the tables user mentioned. 
Since all of the columns in the tables are distinct and do not share common name, you have to use the UNION function in sql query to merge tables into single output table by appending them.
When there are common column names among the tables, you need make them distinctive by merging table's name and column name to create new distinctive name.
"""
"""    
Consider using Common Table Expressions (CTEs) in the following scenarios:
    - When the query involves multiple steps or stages like aggregating, use a CTE to break down the query into simpler, more readable parts.
    - If you are reusing a subset of data multiple times in a single query, use a CTE to define this subset once and reference it as needed.
    - For recursive queries, particularly when dealing with hierarchical data, use a recursive CTE.
    - When preparing data for a pivot table or appending multiple tables, consider using a CTE for intermediate data processing steps.

"""


'    \nConsider using Common Table Expressions (CTEs) in the following scenarios:\n    - When the query involves multiple steps or stages like aggregating, use a CTE to break down the query into simpler, more readable parts.\n    - If you are reusing a subset of data multiple times in a single query, use a CTE to define this subset once and reference it as needed.\n    - For recursive queries, particularly when dealing with hierarchical data, use a recursive CTE.\n    - When preparing data for a pivot table or appending multiple tables, consider using a CTE for intermediate data processing steps.\n\n'

In [63]:
last_five_actions = ""

In [64]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[BaseTool]

    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        #addding file_path
        history_text = ""
        if "history" in kwargs:
            history_messages = kwargs.pop("history")
            for history_message in history_messages:
                history_text += f"{history_message.type}: {history_message.content}\n"
            kwargs["history"] = history_text
        
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        
        # Set the agent_scratchpad variable to that value
        kwargs["last_five_actions"] = last_five_actions
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        # print(formatted)
        return [SystemMessage(content=formatted)]

class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
            # if "Observation:" in llm_output:
                # observation_text = llm_output.split("Observation:")[-1].strip().split("Final Answer")[0]
        if "Final Answer:" in llm_output:
            final_answer_text = llm_output.split("Final Answer:")[-1].strip() 
            output_name_regex = "(<output_name>)\w+(</output_name>)"
            options_regex = "(<list>).+(</list>)"
            output_name_match = re.search(output_name_regex, final_answer_text, re.DOTALL)
            options_match = re.search(options_regex, final_answer_text, re.DOTALL)
            
            if output_name_match:
                output_first_match = output_name_match.group(0)
                output_table_name = output_first_match.split("<output_name>")[1].split("</output_name>")[0]
                final_answer_text = final_answer_text.replace(output_first_match, '')
            else:
                output_table_name = None
                
            if options_match:
                options_first_match = options_match.group(0)
                options_text = options_first_match.split("<list>")[1].split("</list>")[0]
                options = options_text.split(",")
                final_answer_text = final_answer_text.replace(options_first_match, '')
            else:
                options = []

            
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={
                    "output": final_answer_text, 
                    "output_table_name": output_table_name,
                    "options_text": options
                },
                log=llm_output,
            )
        
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [65]:
toolkit = SQLDatabaseToolkit(db=db, llm=AzureChatOpenAI(deployment_name='gpt-4', temperature=0))
tools = toolkit.get_tools()
tools[0].description = """Input to this tool is a detailed and correct SQL query, output is a result from the database. \
The query you are inputing should not be in quotes. \
If the query is not correct, an error message will be returned. \
If an error is returned, rewrite the query, check the query, and try again. \
If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\
Example query is like this: SELECT "Column_1", "Column_2", "Column 3", SUM("Column 4") as column4, FROM table_name GROUP BY "Column_1", "Column_2", "Column 3" LIMIT 10;
"""
tools[1].description = "Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! YOU MUST put input values without any quotes and MUST NOT put any '\n' at the end. YOU MUST follow this example input format: table1, table2, table3"

tool_names = [tool.name for tool in tools]

prompt = CustomPromptTemplate(
    template=sql_helper_prompt_template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)
output_parser = CustomOutputParser()
llm_chain = LLMChain(llm=llm_chat_model, prompt=prompt)
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [66]:
memory = ConversationBufferWindowMemory(memory_key="history", k=10, return_messages=True, output_key="output", input_key="input")

In [67]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    memory=memory,
    return_intermediate_steps=True
)

In [68]:
def add_last_actions(response):
    global last_five_actions
    intermediate_steps = response["intermediate_steps"]
    if len(intermediate_steps) != 0:
        last_action = intermediate_steps[-1][0]
        last_action_log = f"Action tool: {last_action.tool}\nInput: {last_action.tool_input}"
        if len(last_five_actions.split("\n\n")) == 5: 
            last_five_actions = "".join(action_log + "\n\n" for action_log in last_five_actions.split("\n\n")[1:])[:-2]
            last_five_actions = f"{last_five_actions}\n\n{last_action_log}"
        elif last_five_actions == "":
            last_five_actions = last_action_log
        else:
            last_five_actions = f"{last_five_actions}\n\n{last_action_log}"
    print(last_five_actions)

In [69]:
r = agent_executor.invoke({"input": "What are the available tables?"})
add_last_actions(r)
getting_df(r)



> Entering new AgentExecutor chain...
Thought: To get the list of available tables, I need to use the sql_db_list_tables tool.
Action: sql_db_list_tables
Action Input: 

Observation:alamo_cash_foreign_currency, alamo_liabilities, alamo_nt_lookup, alamo_receivable, alamo_securities, alamo_shareholder_register, alamo_ta_history, alamo_total_net_asset, alpha_turnover, output_test_1
I now know the final answer
Final Answer: The available tables are: alamo_cash_foreign_currency, alamo_liabilities, alamo_nt_lookup, alamo_receivable, alamo_securities, alamo_shareholder_register, alamo_ta_history, alamo_total_net_asset, alpha_turnover, output_test_1.

> Finished chain.
Action tool: sql_db_list_tables
Input: 


In [70]:
r = agent_executor.invoke({"input": "Can you show me 10 rows of receivable table?"})
add_last_actions(r)
getting_df(r)



> Entering new AgentExecutor chain...
Thought: To show 10 rows of the receivable table, I need to use the sql_db_query tool with a SELECT statement. I will limit the results to 10 rows.
Action: sql_db_query
Action Input: SELECT * FROM alamo_receivable LIMIT 10;

Observation:DataFrame: {'companyname': ['ALAMO', 'ALAMO', 'ALAMO', 'ALAMO', 'ALAMO', 'ALAMO', 'ALAMO', 'ALAMO', 'ALAMO', 'ALAMO'], 'companyid': ['ALAMO', 'ALAMO', 'ALAMO', 'ALAMO', 'ALAMO', 'ALAMO', 'ALAMO', 'ALAMO', 'ALAMO', 'ALAMO'], 'year': [2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023], 'quarter': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'Fund': ['1736917 - ALAMO GB ALPHA GROWTH FND', '1736917 - ALAMO GB ALPHA GROWTH FND', '1736917 - ALAMO GB ALPHA GROWTH FND', '1736917 - ALAMO GB ALPHA GROWTH FND', '1736917 - ALAMO GB ALPHA GROWTH FND', '1740041 - ALAMO ABSOLUTE RETURN FND', '1740041 - ALAMO ABSOLUTE RETURN FND', '1748760 - ALAMO GB EQTY FND', '1748760 - ALAMO GB EQTY FND', '1748760 - ALAMO GB EQTY FND'], 'Date': ['

,companyname,companyid,year,quarter,Fund,Date,Description,Start,Change,End,For More Information,filename,importdate
0,ALAMO,ALAMO,2023,1,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Accrued Gross Income,0.0,0.0,1843960.23,See Accrued Gross Income,ALAMO - NT data.Receivables202304120444369.xlsx,2023-04-12 16:40:43.775
1,ALAMO,ALAMO,2023,1,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Accrued Gross Withholding,0.0,0.0,-86702.17,See Accrued Net Withholding,ALAMO - NT data.Receivables202304120444369.xlsx,2023-04-12 16:40:43.775
2,ALAMO,ALAMO,2023,1,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Spot Contracts,0.0,0.0,4922221.40,See Spot Contracts Payable/Receivable,ALAMO - NT data.Receivables202304120444369.xlsx,2023-04-12 16:40:43.775
3,ALAMO,ALAMO,2023,1,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Securities Sold,0.0,0.0,417527.74,See Securities Purchased Payable/Sold Receivable,ALAMO - NT data.Receivables202304120444369.xlsx,2023-04-12 16:40:43.775
4,ALAMO,ALAMO,2023,1,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Reclaims,0.0,0.0,3392645.14,See Reclaim Receivable,ALAMO - NT data.Receivables202304120444369.xlsx,2023-04-12 16:40:43.775
5,ALAMO,ALAMO,2023,1,1740041 - ALAMO ABSOLUTE RETURN FND,2023-03-31,Accrued Gross Income,0.0,0.0,14042667.66,See Accrued Gross Income,ALAMO - NT data.Receivables202304120444369.xlsx,2023-04-12 16:40:43.775
6,ALAMO,ALAMO,2023,1,1740041 - ALAMO ABSOLUTE RETURN FND,2023-03-31,Securities Sold,0.0,0.0,-14042667.66,See Securities Purchased Payable/Sold Receivable,ALAMO - NT data.Receivables202304120444369.xlsx,2023-04-12 16:40:43.775
7,ALAMO,ALAMO,2023,1,1748760 - ALAMO GB EQTY FND,2023-03-31,Accrued Gross Income,0.0,0.0,1015934.32,See Accrued Gross Income,ALAMO - NT data.Receivables202304120444369.xlsx,2023-04-12 16:40:43.775
8,ALAMO,ALAMO,2023,1,1748760 - ALAMO GB EQTY FND,2023-03-31,Accrued Gross Withholding,0.0,0.0,-3027.45,See Accrued Net Withholding,ALAMO - NT data.Receivables202304120444369.xlsx,2023-04-12 16:40:43.775
9,ALAMO,ALAMO,2023,1,1748760 - ALAMO GB EQTY FND,2023-03-31,Accrued Gross Reclaim,0.0,0.0,70399.54,See Reclaim Receivable or Accrued Net Withholding,ALAMO - NT data.Receivables202304120444369.xlsx,2023-04-12 16:40:43.775


In [72]:
r = agent_executor.invoke({"input":"Aggregate start and end column values by Description value and group them by Fund, Date and Table type columns"})
add_last_actions(r)
getting_df(r)



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to create a SQL query that aggregates the start and end column values by the Description value and groups them by the Fund, Date, and Table type columns. I will use the SUM function to aggregate the start and end column values. I will also limit the results to 10 rows. 

Action: sql_db_query_checker
Action Input: SELECT "Fund", "Date", "Table type", "Description", SUM("start") as "start", SUM("end") as "end" FROM alamo_receivable GROUP BY "Fund", "Date", "Table type", "Description" LIMIT 10;

Observation:SELECT "Fund", "Date", "Table type", "Description", SUM("start") as "start", SUM("end") as "end" FROM alamo_receivable GROUP BY "Fund", "Date", "Table type", "Description" LIMIT 10;
The query syntax is correct. Now I will execute the query to get the results.

Action: sql_db_query
Action Input: SELECT "Fund", "Date", "Table type", "Description", SUM("start") as "start", SUM("end") as "end" FROM alamo_recei

,Fund,Date,Description,start,end
0,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Accrued Gross Income,0.0,1843960.23
1,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Accrued Gross Withholding,0.0,-86702.17
2,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Spot Contracts,0.0,4922221.40
3,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Securities Sold,0.0,417527.74
4,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Reclaims,0.0,3392645.14
5,1740041 - ALAMO ABSOLUTE RETURN FND,2023-03-31,Accrued Gross Income,0.0,14042667.66
6,1740041 - ALAMO ABSOLUTE RETURN FND,2023-03-31,Securities Sold,0.0,-14042667.66
7,1748760 - ALAMO GB EQTY FND,2023-03-31,Accrued Gross Income,0.0,1015934.32
8,1748760 - ALAMO GB EQTY FND,2023-03-31,Accrued Gross Withholding,0.0,-3027.45
9,1748760 - ALAMO GB EQTY FND,2023-03-31,Accrued Gross Reclaim,0.0,70399.54


In [73]:
r = agent_executor.invoke({"input":"I don't need total_start column. Can you remove it?"})
add_last_actions(r)
getting_df(r) 



> Entering new AgentExecutor chain...
Thought: The user wants to remove the 'total_start' column from the previous query. I will modify the previous SQL query and remove the 'total_start' column.

Action: sql_db_query_checker
Action Input: SELECT "Fund", "Date", "Description", SUM("End") as "end" FROM alamo_receivable GROUP BY "Fund", "Date", "Description" LIMIT 10;

Observation:SELECT "Fund", "Date", "Description", SUM("End") as "end" FROM alamo_receivable GROUP BY "Fund", "Date", "Description" LIMIT 10;
The query has been checked and it is correct. Now I will execute it.

Action: sql_db_query
Action Input: SELECT "Fund", "Date", "Description", SUM("End") as "end" FROM alamo_receivable GROUP BY "Fund", "Date", "Description" LIMIT 10;

Observation:DataFrame: {'Fund': ['1736917 - ALAMO GB ALPHA GROWTH FND', '1736917 - ALAMO GB ALPHA GROWTH FND', '1736917 - ALAMO GB ALPHA GROWTH FND', '1736917 - ALAMO GB ALPHA GROWTH FND', '1736917 - ALAMO GB ALPHA GROWTH FND', '1740041 - ALAMO ABSOLUT

,Fund,Date,Description,end
0,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Accrued Gross Income,1843960.23
1,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Accrued Gross Withholding,-86702.17
2,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Spot Contracts,4922221.40
3,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Securities Sold,417527.74
4,1736917 - ALAMO GB ALPHA GROWTH FND,2023-03-31,Reclaims,3392645.14
5,1740041 - ALAMO ABSOLUTE RETURN FND,2023-03-31,Accrued Gross Income,14042667.66
6,1740041 - ALAMO ABSOLUTE RETURN FND,2023-03-31,Securities Sold,-14042667.66
7,1748760 - ALAMO GB EQTY FND,2023-03-31,Accrued Gross Income,1015934.32
8,1748760 - ALAMO GB EQTY FND,2023-03-31,Accrued Gross Reclaim,70399.54
9,1748760 - ALAMO GB EQTY FND,2023-03-31,Reclaims,2743421.44


In [74]:
r = agent_executor.invoke({"input":"Give me output table from this and liabilities table"})
add_last_actions(r)
getting_df(r) 



> Entering new AgentExecutor chain...
Thought: The user wants to create an output table from the last result and the liabilities table. I need to ask the user for the name of the output table.

Final Answer: What would you like to name the output table?

> Finished chain.
Action tool: sql_db_list_tables
Input: 

Action tool: sql_db_query
Input: SELECT * FROM alamo_receivable LIMIT 10;

Action tool: sql_db_query
Input: SELECT "Fund", "Date", "Description", SUM("Start") as "start", SUM("End") as "end" FROM alamo_receivable GROUP BY "Fund", "Date", "Description" LIMIT 10;

Action tool: sql_db_query
Input: SELECT "Fund", "Date", "Description", SUM("End") as "end" FROM alamo_receivable GROUP BY "Fund", "Date", "Description" LIMIT 10;


In [75]:
r = agent_executor.invoke({"input":"output_table_2"})
add_last_actions(r)
getting_df(r) 



> Entering new AgentExecutor chain...
Thought: The user wants to create an output table from the last result and the liabilities table. I need to append these two tables. I will first check the schema of the liabilities table to understand its structure. Then I will create a SQL query to append these two tables. 

Action: sql_db_schema
Action Input: alamo_liabilities

Observation:
CREATE TABLE alamo_liabilities (
	companyname VARCHAR(255), 
	companyid VARCHAR(255), 
	year DECIMAL(38, 0), 
	quarter DECIMAL(38, 0), 
	fund VARCHAR(255), 
	date VARCHAR(255), 
	description VARCHAR(255), 
	"START" FLOAT, 
	change FLOAT, 
	end FLOAT, 
	for_more_information VARCHAR(255), 
	filename VARCHAR(255), 
	importdate TIMESTAMP_NTZ
)

/*
3 rows from alamo_liabilities table:
companyname	companyid	year	quarter	fund	date	description	START	change	end	for_more_information	filename	importdate
ALAMO	ALAMO	2023	1	1736917 - ALAMO GB ALPHA GROWTH FND	2023-03-31	Payables	None	None	None	None	ALAMO - NT data.Liabi

ProgrammingError: (snowflake.connector.errors.ProgrammingError) 002002 (42710): SQL compilation error:
Object 'OUTPUT_TABLE_2' already exists.
[SQL: CREATE TABLE output_table_2 AS (
SELECT "Fund", "Date", "Description", SUM("End") as "end", 'ALAMO_RECEIVABLE' as "Source table" FROM alamo_receivable GROUP BY "Fund", "Date", "Description"
UNION ALL
SELECT fund, date, description, end, 'ALAMO_LIABILITIES' as "Source table" FROM alamo_liabilities
);]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [54]:
r = agent_executor.invoke({"input":"ouput_table_1"})
add_last_actions(r)
getting_df(r) 



> Entering new AgentExecutor chain...
Thought: The user wants to create an output table from the last result and the 'liabilities' table. The name of the output table should be 'ouput_table_1'. I need to check the schema of the 'liabilities' table to ensure that the columns match with the last result. If they don't, I'll need to fill in the missing columns with NULL values.

Action: sql_db_schema
Action Input: alamo_liabilities

Observation:
CREATE TABLE alamo_liabilities (
	companyname VARCHAR(255), 
	companyid VARCHAR(255), 
	year DECIMAL(38, 0), 
	quarter DECIMAL(38, 0), 
	fund VARCHAR(255), 
	date VARCHAR(255), 
	description VARCHAR(255), 
	"START" FLOAT, 
	change FLOAT, 
	end FLOAT, 
	for_more_information VARCHAR(255), 
	filename VARCHAR(255), 
	importdate TIMESTAMP_NTZ
)

/*
3 rows from alamo_liabilities table:
companyname	companyid	year	quarter	fund	date	description	START	change	end	for_more_information	filename	importdate
ALAMO	ALAMO	2023	1	1736917 - ALAMO GB ALPHA GROWTH FND

In [295]:
r = agent_executor.invoke({"input":"Can you turn 'Table type' values into seperate columns from this result?"})
add_last_actions(r)
getting_df(r) 



> Entering new AgentExecutor chain...
Thought: The user wants to pivot the table on the 'Table type' column. To do this, I need to know the distinct values in the 'Table type' column. I will use the last SQL query to get the distinct values in the 'Table type' column.
Action: sql_db_query
Action Input: SELECT DISTINCT "Table type" FROM joined_balance;

Observation:DataFrame: {'Table type': ['Receivables', 'Liabilities', 'Total net assets']}
The distinct values in the 'Table type' column are 'Receivables', 'Liabilities', and 'Total net assets'. I will now create a pivot table with these values as separate columns. Since the user didn't specify an aggregation function, I will use SUM as the default function to aggregate the 'Total_End' column.
Action: sql_db_query
Action Input: SELECT "Fund", "Date", SUM(IF("Table type" = 'Receivables', "Total_End", NULL)) AS "Receivables", SUM(IF("Table type" = 'Liabilities', "Total_End", NULL)) AS "Liabilities", SUM(IF("Table type" = 'Total net asset

,Fund,Date,Receivables,Liabilities,Total net assets
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,1.888321e+07,-3318115.24,8.900892e+08
1,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,0.000000e+00,-750321.31,7.863990e+08
2,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,0.000000e+00,-247285.98,2.150252e+08
3,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,1.083894e+07,-62058.94,1.143294e+09
4,1745443 - LCIV REAL RETURN FND,30 Dec 22,0.000000e+00,-27370.62,1.743970e+08
5,1748760 - LCIV GB EQTY FND,30 Dec 22,7.153698e+06,-2287745.09,5.565896e+08
6,1748761 - LCIV UK EQTY FND,30 Dec 22,8.742849e+04,0.00,0.000000e+00
7,1751062 - LCIV SUSTAINABLE EQTY FND,30 Dec 22,4.589948e+07,-4116941.68,1.238336e+09
8,1762287 - LCIV GLOBAL BOND FUND,30 Dec 22,2.632425e+08,-77368078.13,6.509100e+08
9,1762300 - LCIV MAC INC COLL FUND,30 Dec 22,NaN,NaN,1.740521e+06


In [296]:
r = agent_executor.invoke({"input":"Produce output table from this data and cash table"})
add_last_actions(r)
getting_df(r)



> Entering new AgentExecutor chain...
Thought: The user wants to create an output table from the last result and the 'cash' table. I need to ask the user for the name of the output table.

Final Answer: What would you like to name the output table?

> Finished chain.
Action tool: sql_db_query
Input: SELECT * FROM joined_balance LIMIT 10;

Action tool: sql_db_query
Input: SELECT "Fund", "Date", "Table type", SUM("Start") as Total_Start, SUM("End") as Total_End FROM joined_balance GROUP BY "Fund", "Date", "Table type" LIMIT 10;

Action tool: sql_db_query
Input: SELECT "Fund", "Date", "Table type", SUM("End") as Total_End FROM joined_balance GROUP BY "Fund", "Date", "Table type" LIMIT 10;

Action tool: sql_db_query
Input: SELECT "Fund", "Date", SUM(CASE WHEN "Table type" = 'Receivables' THEN "End" ELSE NULL END) AS "Receivables", SUM(CASE WHEN "Table type" = 'Liabilities' THEN "End" ELSE NULL END) AS "Liabilities", SUM(CASE WHEN "Table type" = 'Total net assets' THEN "End" ELSE NULL END

In [297]:
r = agent_executor.invoke({"input":"output_test_7 would be great"})
add_last_actions(r)
getting_df(r)



> Entering new AgentExecutor chain...
Thought: The user wants to create an output table named 'output_test_7' from the last result and the 'cash' table. I need to check the schema of the 'cash' table first to know its structure and columns. 
Action: sql_db_schema
Action Input: cash

Observation:
CREATE TABLE cash (
	"Valuation Date" DATE, 
	"Account ID (FA)" VARCHAR(255), 
	"Account ID (FA)/Name" VARCHAR(255), 
	"Account ID (Custody)" VARCHAR(255), 
	"Security Description (Short)" VARCHAR(255), 
	"Security Number (Full)" VARCHAR(255), 
	"Issue Currency" VARCHAR(255), 
	"Long/Short Indicator" VARCHAR(255), 
	"Country of Risk" VARCHAR(255)
)

/*
3 rows from cash table:
Valuation Date	Account ID (FA)	Account ID (FA)/Name	Account ID (Custody)	Security Description (Short)	Security Number (Full)	Issue Currency	Long/Short Indicator	Country of Risk
2021-09-30	1736917	1736917 - LCIV GB ALPHA GROWTH FND	LLGQ01	UNITED STATES DOLLAR	USD	USD	L	US
2021-09-30	1736917	1736917 - LCIV GB ALPHA GROWTH 

,status
0,Table OUTPUT_TEST_7 successfully created.


In [144]:
r = agent_executor.invoke({"input":"output_joined_balance"})
add_last_actions(r)
getting_df(r)



> Entering new AgentExecutor chain...
Thought: The user wants to create a new output table from the last query. I need to create a new table in the database with the name 'output_joined_balance'. I will use the CREATE TABLE AS SELECT statement to create a new table from the result of a query. I will use the last query I executed to create this new table.
Action: sql_db_query_checker
Action Input: CREATE TABLE output_joined_balance AS SELECT "Fund", "Date", "Table type", SUM("Start") as Total_Start, SUM("End") as Total_End FROM joined_balance GROUP BY "Fund", "Date", "Table type";

Observation:CREATE TABLE output_joined_balance AS SELECT "Fund", "Date", "Table type", SUM("Start") as Total_Start, SUM("End") as Total_End FROM joined_balance GROUP BY "Fund", "Date", "Table type";
The query has been checked and is correct. Now I can execute it to create the new table.
Action: sql_db_query
Action Input: CREATE TABLE output_joined_balance AS SELECT "Fund", "Date", "Table type", SUM("Start")

In [145]:
r = agent_executor.invoke({"input":"Give me the first 10 rows of cash table"})
add_last_actions(r)
getting_df(r)



> Entering new AgentExecutor chain...
Thought: I need to use the sql_db_query tool to get the first 10 rows of the cash table.
Action: sql_db_query
Action Input: SELECT * FROM cash LIMIT 10;

Observation:DataFrame: {'Valuation Date': [datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30)], 'Account ID (FA)': ['1736917', '1736917', '1736917', '1736917', '1748761', '1748760', '1748760', '1748760', '1750023', '1751063'], 'Account ID (FA)/Name': ['1736917 - LCIV GB ALPHA GROWTH FND', '1736917 - LCIV GB ALPHA GROWTH FND', '1736917 - LCIV GB ALPHA GROWTH FND', '1736917 - LCIV GB ALPHA GROWTH FND', '1748761 - LCIV UK EQTY FND', '1748760 - LCIV GB EQTY FND', '1748760 - LCIV GB EQTY FND', '1748760 - LCIV GB EQTY FND', '1750023 - LCIV GB EQ FOCUS FND', '1751063 - LCIV EQTY

ValueError: malformed node or string on line 1: <ast.Call object at 0x16515fd90>

In [148]:
r = agent_executor.invoke({"input":"What are the available tables?"})
add_last_actions(r)
getting_df(r)



> Entering new AgentExecutor chain...
Thought: To find out the available tables, I need to use the sql_db_list_tables tool.
Action: sql_db_list_tables
Action Input: ""

Observation:balance, cash, investment, joined_balance, output_final, output_test_2, turnover
I now know the final answer
Final Answer: The available tables are: balance, cash, investment, joined_balance, output_final, output_test_2, turnover.

> Finished chain.
Action tool: sql_db_query
Input: SELECT "Fund", "Date", "Description", "Start", "Change", "End", "Table type" FROM joined_balance LIMIT 10;

Action tool: sql_db_query
Input: SELECT "Fund", "Date", "Table type", SUM("Start") as Total_Start, SUM("End") as Total_End FROM joined_balance GROUP BY "Fund", "Date", "Table type" LIMIT 10;

Action tool: sql_db_query
Input: CREATE TABLE output_joined_balance AS SELECT "Fund", "Date", "Table type", SUM("Start") as Total_Start, SUM("End") as Total_End FROM joined_balance GROUP BY "Fund", "Date", "Table type";

Action tool: 

In [124]:
r = agent_executor.invoke({"input":"Can you give me joined_balance table data by aggregating start and end column values by Table type value and grouping them by Fund, Date and Table type columns?"})
add_last_actions(r)
getting_df(r)



> Entering new AgentExecutor chain...

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct sql query to run on snowflake, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, ALWAYS limit your query to at most 10 results.
User will interact with you while giving you the filters he/she wants. 

User might ask to pivot a table. So here's the rule for this situation:
    - You need to know the distinct values in the asked column to create the pivot table
    - If user didn't request specifically, use SUM function to aggregate.

User might ask you to append multiple tables or produce output table from them. So here's the rules for this situation:
    - You have to append multiple tables so that output must contain all of the columns in the tables user mentioned. 
    - Since all of the columns in the tables are distinct and do not share commo

KeyboardInterrupt: 

In [289]:
r = agent_executor.invoke({"input":"I do not need tot of start column from this result? Can you remove it?"})
add_last_actions(r)
getting_df(r)




> Entering new AgentExecutor chain...
Thought: The user wants to remove the total of the start column from the previous result. I will modify the previous SQL query by removing the aggregation of the start column.
Action: sql_db_query_checker
Action Input: SELECT "Fund", "Date", "Table type", SUM("End") as Total_End FROM joined_balance GROUP BY "Fund", "Date", "Table type" ORDER BY "Date" LIMIT 10;

Observation:SELECT "Fund", "Date", "Table type", SUM("End") as Total_End 
FROM joined_balance 
GROUP BY "Fund", "Date", "Table type" 
ORDER BY "Date" 
LIMIT 10;
The query syntax is correct. Now I can execute the query to get the desired result.
Action: sql_db_query
Action Input: SELECT "Fund", "Date", "Table type", SUM("End") as Total_End FROM joined_balance GROUP BY "Fund", "Date", "Table type" ORDER BY "Date" LIMIT 10;

Observation:DF: {'Fund': ['1736917 - LCIV GB ALPHA GROWTH FND', '1736917 - LCIV GB ALPHA GROWTH FND', '1736917 - LCIV GB ALPHA GROWTH FND', '1736938 - LCIV DIVERSIFD GRO

,Fund,Date,Table type,total_end
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Receivables,1.888321e+07
1,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Liabilities,-3.318115e+06
2,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Total net assets,8.900892e+08
3,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Liabilities,-7.503213e+05
4,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Total net assets,7.863990e+08
5,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Receivables,0.000000e+00
6,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Receivables,0.000000e+00
7,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Liabilities,-2.472860e+05
8,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Total net assets,2.150252e+08
9,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,Liabilities,-6.205894e+04


In [290]:
r = agent_executor.invoke({"input":"Can you produce output table from this result and cash table? Name it output_test_1"})
add_last_actions(r)
getting_df(r)




> Entering new AgentExecutor chain...
Thought: The user wants to create an output table from the result of the last query and the 'cash' table. I need to check the schema of the 'cash' table to understand its structure and columns. Then, I can create a new table by appending the result of the last query and the 'cash' table. 

Action: sql_db_schema
Action Input: cash

Observation:
CREATE TABLE cash (
	"Valuation Date" DATE, 
	"Account ID (FA)" VARCHAR(255), 
	"Account ID (FA)/Name" VARCHAR(255), 
	"Account ID (Custody)" VARCHAR(255), 
	"Security Description (Short)" VARCHAR(255), 
	"Security Number (Full)" VARCHAR(255), 
	"Issue Currency" VARCHAR(255), 
	"Long/Short Indicator" VARCHAR(255), 
	"Country of Risk" VARCHAR(255)
)

/*
3 rows from cash table:
Valuation Date	Account ID (FA)	Account ID (FA)/Name	Account ID (Custody)	Security Description (Short)	Security Number (Full)	Issue Currency	Long/Short Indicator	Country of Risk
2021-09-30	1736917	1736917 - LCIV GB ALPHA GROWTH FND	LLGQ

ValueError: Could not parse LLM output: `The 'cash' table has the following columns: "Valuation Date", "Account ID (FA)", "Account ID (FA)/Name", "Account ID (Custody)", "Security Description (Short)", "Security Number (Full)", "Issue Currency", "Long/Short Indicator", "Country of Risk". 

The result of the last query has the following columns: "Fund", "Date", "Table type", "Total_End". 

To append these two tables, I need to make sure they have the same number of columns. Since they don't, I will fill in the missing columns with NULL values. 

The new output table will have the following structure:

| Fund | Date | Table type | Total_End | Valuation Date | Account ID (FA) | Account ID (FA)/Name | Account ID (Custody) | Security Description (Short) | Security Number (Full) | Issue Currency | Long/Short Indicator | Country of Risk |

Is this structure correct?`

In [267]:
r = agent_executor.invoke({"input":"What's the last query you used?"})
getting_df(r)
add_last_actions(r)



> Entering new AgentExecutor chain...

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct sql query to run on snowflake, then look at the results of the query and return the answer.
When producing query, DO NOT put quotes around the whole query.
Since you are working with snowflake, you have to get table names without the quotes.
Unless the user specifies a specific number of examples they wish to obtain, ALWAYS limit your query to at most 10 results.
You can order the results by a relevant column to return the most interesting examples in the database.
User will interact with you while giving you the filters he/she wants. 
If you user recalls the filters they asked by, you have to recall which query or queries they are asking. If you need help, ask them to confirm. 
User might ask to produce a table after several steps. Make sure to apply all the necessary filters they asked before.
If user asks to do anything with the 

### past attempts , I'm saving up for the analysis of the results 

In [26]:
r = agent_executor.invoke({"input":"Can you give me the first 10 rows of joined_balance data?"})



> Entering new AgentExecutor chain...
""  action input
Thought: To get the first 10 rows of the 'joined_balance' table, I need to write a SQL query. But first, I should check if the 'joined_balance' table exists in the database.

Action: sql_db_list_tables
Action Input: ""

Observation:balance, cash, investment, joined_balance, joined_output_table, output_test_1, output_test_6, turnover
action:  tool='sql_db_list_tables' tool_input='' log='Thought: To get the first 10 rows of the \'joined_balance\' table, I need to write a SQL query. But first, I should check if the \'joined_balance\' table exists in the database.\n\nAction: sql_db_list_tables\nAction Input: ""'
SELECT * FROM joined_balance LIMIT 10;  action input
The 'joined_balance' table does exist in the database. Now, I can write a SQL query to get the first 10 rows of the 'joined_balance' table.

Action: sql_db_query_checker
Action Input: SELECT * FROM joined_balance LIMIT 10;

Observation:SELECT * FROM joined_balance LIMIT 10;

In [127]:
getting_df(r)

,Fund,Date,Description,Start,Change,End,More Information,Table type
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Accrued Gross Income,1.100336e+06,1.085218e+06,2.185554e+06,See Accrued Gross Income,Receivables
1,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Payables,NaN,NaN,NaN,None,Liabilities
2,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Accrued Expenses,-3.886694e+06,9.415130e+05,-2.945181e+06,See Accrued Expenses,Liabilities
3,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Accrued Capital Expenses,-2.902319e+05,-8.256802e+04,-3.727999e+05,None,Liabilities
4,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Spot Contracts,-3.636267e+04,3.636267e+04,0.000000e+00,See Spot Contracts Payable/Receivable,Liabilities
5,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Income,-4.583996e+06,4.583996e+06,0.000000e+00,See Income Payable,Liabilities
6,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Unrealized Loss on Forward Contracts,0.000000e+00,-1.341400e+02,-1.341400e+02,See Unrealized G/L on Currency Contracts,Liabilities
7,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Total Net Assets,1.229889e+09,-3.397998e+08,8.900892e+08,None,Total net assets
8,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Prepaid Expenses,1.728732e+06,1.396337e+05,1.868366e+06,See Prepaid Expenses,Receivables
9,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Reclaims,3.265934e+06,6.580191e+04,3.331735e+06,See Reclaim Receivable,Receivables


In [128]:
r = agent_executor.invoke({"input":"Can you give me joined_balance table data by aggregating start and end column values by Table type value and grouping them by Fund, Date and Table type columns?"})



> Entering new AgentExecutor chain...
SELECT Fund, Date, Table_type, SUM(start), SUM(end) FROM joined_balance GROUP BY Fund, Date, Table_type LIMIT 10;  action input
Thought: To answer this question, I need to write a SQL query that aggregates the 'start' and 'end' column values by 'Table type' value and groups them by 'Fund', 'Date' and 'Table type' columns from the 'joined_balance' table. I will use the SUM function to aggregate the 'start' and 'end' column values. 

Action: sql_db_query_checker
Action Input: SELECT Fund, Date, Table_type, SUM(start), SUM(end) FROM joined_balance GROUP BY Fund, Date, Table_type LIMIT 10;

Observation:SELECT Fund, Date, Table_type, SUM(start), SUM(end) FROM joined_balance GROUP BY Fund, Date, Table_type LIMIT 10;
SELECT Fund, Date, Table_type, SUM(start), SUM(end) FROM joined_balance GROUP BY Fund, Date, Table_type LIMIT 10;  action input
The query syntax is correct. Now I can execute the query to get the required data.

Action: sql_db_query
Action 

In [129]:
getting_df(r)

,Fund,Date,Table type,"SUM(""START"")","SUM(""END"")"
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Receivables,2.208791e+07,1.888321e+07
1,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Liabilities,-8.797284e+06,-3.318115e+06
2,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Total net assets,1.229889e+09,8.900892e+08
3,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Liabilities,-5.937032e+06,-7.503213e+05
4,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Total net assets,7.727695e+08,7.863990e+08
5,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Receivables,0.000000e+00,0.000000e+00
6,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Receivables,2.118886e+06,0.000000e+00
7,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Liabilities,-8.974092e+05,-2.472860e+05
8,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Total net assets,2.117766e+08,2.150252e+08
9,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,Liabilities,-3.629060e+05,-6.205894e+04


In [130]:
r = agent_executor.invoke({"input":"I do not need sum of start column from this result? Can you remove it?"})



> Entering new AgentExecutor chain...
SELECT Fund, Date, Table_type, SUM(end) FROM joined_balance GROUP BY Fund, Date, Table_type;  action input
Thought: The user wants to remove the sum of the start column from the previous result. I need to modify the previous SQL query to exclude the sum of the start column. 

Action: sql_db_query_checker
Action Input: SELECT Fund, Date, Table_type, SUM(end) FROM joined_balance GROUP BY Fund, Date, Table_type;

Observation:SELECT Fund, Date, Table_type, SUM(end) FROM joined_balance GROUP BY Fund, Date, Table_type;
SELECT Fund, Date, Table_type, SUM(end) FROM joined_balance GROUP BY Fund, Date, Table_type;  action input
The query is correct. Now I can execute it to get the desired result.

Action: sql_db_query
Action Input: SELECT Fund, Date, Table_type, SUM(end) FROM joined_balance GROUP BY Fund, Date, Table_type;

Observation:Error: (snowflake.connector.errors.ProgrammingError) 000904 (42000): SQL compilation error: error line 1 at position 7
inv

In [131]:
getting_df(r)

,Fund,Date,Table type,"SUM(""END"")"
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Receivables,1.888321e+07
1,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Liabilities,-3.318115e+06
2,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Total net assets,8.900892e+08
3,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Liabilities,-7.503213e+05
4,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Total net assets,7.863990e+08
5,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Receivables,0.000000e+00
6,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Receivables,0.000000e+00
7,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Liabilities,-2.472860e+05
8,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Total net assets,2.150252e+08
9,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,Liabilities,-6.205894e+04


In [132]:
r = agent_executor.invoke({"input":"Can you turn 'Table type' values into seperate columns from this result?"})



> Entering new AgentExecutor chain...
SELECT DISTINCT "Table type" FROM joined_balance LIMIT 10;  action input
Thought: The user wants to pivot the 'Table type' values into separate columns. This can be achieved using a PIVOT operation in SQL. However, I need to know the unique values in the 'Table type' column to create the pivot table. I will first run a query to get the distinct 'Table type' values from the 'joined_balance' table. 

Action: sql_db_query
Action Input: SELECT DISTINCT "Table type" FROM joined_balance LIMIT 10;

Observation:DF: {'Table type': ['Receivables', 'Liabilities', 'Total net assets']}
SELECT "Fund", "Date", MAX(IF("Table type" = 'Receivables', "end", NULL)) AS "Receivables", MAX(IF("Table type" = 'Liabilities', "end", NULL)) AS "Liabilities", MAX(IF("Table type" = 'Total net assets', "end", NULL)) AS "Total net assets" FROM joined_balance GROUP BY "Fund", "Date";  action input
The unique values in the 'Table type' column are 'Receivables', 'Liabilities', and

In [133]:
getting_df(r)

,Fund,Date,Receivables,Liabilities,Total net assets
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,7.996314e+06,0.0,8.900892e+08
1,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,0.000000e+00,0.0,7.863990e+08
2,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,0.000000e+00,0.0,2.150252e+08
3,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,1.083894e+07,0.0,1.143294e+09
4,1745443 - LCIV REAL RETURN FND,30 Dec 22,0.000000e+00,-96.5,1.743970e+08
5,1748760 - LCIV GB EQTY FND,30 Dec 22,2.723829e+06,0.0,5.565896e+08
6,1748761 - LCIV UK EQTY FND,30 Dec 22,8.715891e+04,0.0,0.000000e+00
7,1751062 - LCIV SUSTAINABLE EQTY FND,30 Dec 22,4.258751e+07,0.0,1.238336e+09
8,1762287 - LCIV GLOBAL BOND FUND,30 Dec 22,2.537348e+08,0.0,6.509100e+08
9,1762300 - LCIV MAC INC COLL FUND,30 Dec 22,NaN,NaN,1.740521e+06


### after I changed the prompt to use SUM function to aggregate

In [322]:
r = agent_executor.invoke({"input":"Can you give me the first 10 rows of joined_balance data?"})
getting_df(r)



> Entering new AgentExecutor chain...
""  action input
Thought: To get the first 10 rows of the 'joined_balance' table, I need to write a SQL query. However, before that, I should check if the 'joined_balance' table exists in the database.

Action: sql_db_list_tables
Action Input: ""

Observation:balance, cash, investment, joined_balance, joined_balance_output, joined_balance_transformed, output_test_1, output_test_6, turnover
SELECT * FROM joined_balance LIMIT 10;  action input
The 'joined_balance' table does exist in the database. Now, I can write a SQL query to fetch the first 10 rows from this table.

Action: sql_db_query_checker
Action Input: SELECT * FROM joined_balance LIMIT 10;

Observation:SELECT * FROM joined_balance LIMIT 10;
SELECT * FROM joined_balance LIMIT 10;  action input
The SQL query is correct. Now, I can execute this query to fetch the first 10 rows from the 'joined_balance' table.

Action: sql_db_query
Action Input: SELECT * FROM joined_balance LIMIT 10;

Observ

,Fund,Date,Description,Start,Change,End,More Information,Table type
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Accrued Gross Income,1.100336e+06,1.085218e+06,2.185554e+06,See Accrued Gross Income,Receivables
1,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Payables,NaN,NaN,NaN,None,Liabilities
2,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Accrued Expenses,-3.886694e+06,9.415130e+05,-2.945181e+06,See Accrued Expenses,Liabilities
3,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Accrued Capital Expenses,-2.902319e+05,-8.256802e+04,-3.727999e+05,None,Liabilities
4,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Spot Contracts,-3.636267e+04,3.636267e+04,0.000000e+00,See Spot Contracts Payable/Receivable,Liabilities
5,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Income,-4.583996e+06,4.583996e+06,0.000000e+00,See Income Payable,Liabilities
6,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Unrealized Loss on Forward Contracts,0.000000e+00,-1.341400e+02,-1.341400e+02,See Unrealized G/L on Currency Contracts,Liabilities
7,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Total Net Assets,1.229889e+09,-3.397998e+08,8.900892e+08,None,Total net assets
8,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Prepaid Expenses,1.728732e+06,1.396337e+05,1.868366e+06,See Prepaid Expenses,Receivables
9,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Reclaims,3.265934e+06,6.580191e+04,3.331735e+06,See Reclaim Receivable,Receivables


In [323]:
r = agent_executor.invoke({"input":"Can you give me joined_balance table data by aggregating start and end column values by Table type value and grouping them by Fund, Date and Table type columns?"})
getting_df(r)



> Entering new AgentExecutor chain...
joined_balance  action input
Thought: To answer this question, I need to create a SQL query that aggregates the 'start' and 'end' column values by 'Table type' value and groups them by 'Fund', 'Date' and 'Table type' columns. I will use the 'joined_balance' table for this query. But first, I need to check the schema of the 'joined_balance' table to make sure these columns exist.

Action: sql_db_schema
Action Input: joined_balance

Observation:
CREATE TABLE joined_balance (
	"Fund" VARCHAR(255), 
	"Date" VARCHAR(255), 
	"Description" VARCHAR(255), 
	"Start" FLOAT, 
	"Change" FLOAT, 
	"End" FLOAT, 
	"More Information" VARCHAR(255), 
	"Table type" VARCHAR(255)
)

/*
3 rows from joined_balance table:
Fund	Date	Description	Start	Change	End	More Information	Table type
1736917 - LCIV GB ALPHA GROWTH FND	30 Dec 22	Accrued Gross Income	1100336.05	1085218.41	2185554.46	See Accrued Gross Income	Receivables
1736917 - LCIV GB ALPHA GROWTH FND	30 Dec 22	Payabl

,Fund,Date,Table type,total_start,total_end
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Receivables,2.208791e+07,1.888321e+07
1,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Liabilities,-8.797284e+06,-3.318115e+06
2,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Total net assets,1.229889e+09,8.900892e+08
3,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Liabilities,-5.937032e+06,-7.503213e+05
4,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Total net assets,7.727695e+08,7.863990e+08
5,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Receivables,0.000000e+00,0.000000e+00
6,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Receivables,2.118886e+06,0.000000e+00
7,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Liabilities,-8.974092e+05,-2.472860e+05
8,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Total net assets,2.117766e+08,2.150252e+08
9,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,Liabilities,-3.629060e+05,-6.205894e+04


In [324]:
r = agent_executor.invoke({"input":"I do not need total of start column from this result? Can you remove it?"})
getting_df(r)



> Entering new AgentExecutor chain...
SELECT "Fund", "Date", "Table type", SUM("end") as end FROM joined_balance GROUP BY "Fund", "Date", "Table type" LIMIT 10;  action input
Thought: The user wants to remove the total of the start column from the previous result. I need to modify the previous SQL query by removing the aggregation of the start column. 

Action: sql_db_query_checker
Action Input: SELECT "Fund", "Date", "Table type", SUM("end") as end FROM joined_balance GROUP BY "Fund", "Date", "Table type" LIMIT 10;

Observation:SELECT "Fund", "Date", "Table type", SUM("end") as end FROM joined_balance GROUP BY "Fund", "Date", "Table type" LIMIT 10;
SELECT "Fund", "Date", "Table type", SUM("end") as end FROM joined_balance GROUP BY "Fund", "Date", "Table type" LIMIT 10;  action input
The query has been checked and it is correct. Now I can execute it to get the desired result.

Action: sql_db_query
Action Input: SELECT "Fund", "Date", "Table type", SUM("end") as end FROM joined_balanc

,Fund,Date,Table type,end
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Receivables,1.888321e+07
1,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Liabilities,-3.318115e+06
2,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Total net assets,8.900892e+08
3,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Liabilities,-7.503213e+05
4,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Total net assets,7.863990e+08
5,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Receivables,0.000000e+00
6,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Receivables,0.000000e+00
7,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Liabilities,-2.472860e+05
8,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Total net assets,2.150252e+08
9,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,Liabilities,-6.205894e+04


In [325]:
r = agent_executor.invoke({"input":"Can you turn 'Table type' values into seperate columns from this result?"})
getting_df(r)



> Entering new AgentExecutor chain...
SELECT DISTINCT "Table type" FROM (SELECT "Fund", "Date", "Table type", SUM("end") as end FROM joined_balance GROUP BY "Fund", "Date", "Table type") as subquery;  action input
Thought: To pivot the table, I need to know the distinct values in the 'Table type' column from the last result. I will use the last query I produced, but this time I will select only 'Table type' column and use DISTINCT to get unique values. 

Action: sql_db_query
Action Input: SELECT DISTINCT "Table type" FROM (SELECT "Fund", "Date", "Table type", SUM("end") as end FROM joined_balance GROUP BY "Fund", "Date", "Table type") as subquery;

Observation:Error: (snowflake.connector.errors.ProgrammingError) 000904 (42000): SQL compilation error: error line 1 at position 76
invalid identifier '"end"'
[SQL: SELECT DISTINCT "Table type" FROM (SELECT "Fund", "Date", "Table type", SUM("end") as end FROM joined_balance GROUP BY "Fund", "Date", "Table type") as subquery;]
(Background o

,Fund,Date,receivables,liabilities,total_net_assets
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,1.888321e+07,-3318115.24,8.900892e+08
1,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,0.000000e+00,-750321.31,7.863990e+08
2,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,0.000000e+00,-247285.98,2.150252e+08
3,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,1.083894e+07,-62058.94,1.143294e+09
4,1745443 - LCIV REAL RETURN FND,30 Dec 22,0.000000e+00,-27370.62,1.743970e+08
5,1748760 - LCIV GB EQTY FND,30 Dec 22,7.153698e+06,-2287745.09,5.565896e+08
6,1748761 - LCIV UK EQTY FND,30 Dec 22,8.742849e+04,0.00,0.000000e+00
7,1751062 - LCIV SUSTAINABLE EQTY FND,30 Dec 22,4.589948e+07,-4116941.68,1.238336e+09
8,1762287 - LCIV GLOBAL BOND FUND,30 Dec 22,2.632425e+08,-77368078.13,6.509100e+08
9,1762300 - LCIV MAC INC COLL FUND,30 Dec 22,NaN,NaN,1.740521e+06


In [327]:
r = agent_executor.invoke({"input":"Can you give me output table from this result under the name of 'joined_output_table'?"})



> Entering new AgentExecutor chain...
CREATE TABLE joined_output_table AS (SELECT * FROM (SELECT Fund, Date, Table_type, end FROM joined_balance GROUP BY Fund, Date, Table_type));  action input
Thought: The user wants to create an output table from the last result. I need to create a SQL query to create a new table from the last result. The name of the new table should be 'joined_output_table'. 

Action: sql_db_query_checker
Action Input: CREATE TABLE joined_output_table AS (SELECT * FROM (SELECT Fund, Date, Table_type, end FROM joined_balance GROUP BY Fund, Date, Table_type));

Observation:CREATE TABLE joined_output_table AS (SELECT * FROM (SELECT Fund, Date, Table_type, end FROM joined_balance GROUP BY Fund, Date, Table_type));
CREATE TABLE joined_output_table AS (SELECT * FROM (SELECT Fund, Date, Table_type, end FROM joined_balance GROUP BY Fund, Date, Table_type));  action input
The query has been checked and it is correct. Now I can execute it to create the new table.

Action: s

In [315]:
r = agent_executor.invoke({"input":"joined_balance_output"})



> Entering new AgentExecutor chain...
CREATE TABLE joined_balance_output AS (SELECT * FROM (SELECT Fund, Date, Table type, SUM(end) as end FROM joined_balance GROUP BY Fund, Date, Table type) PIVOT (SUM(end) FOR Table type IN ('Type1', 'Type2', 'Type3')) LIMIT 10);  action input
Thought: I need to create an output table from the latest query result. The user has provided the name for the output table as 'joined_balance_output'. I will use the CREATE TABLE AS SELECT statement to create a new table from the result of the query. But before that, I need to check the query to make sure it's correct.
Action: sql_db_query_checker
Action Input: CREATE TABLE joined_balance_output AS (SELECT * FROM (SELECT Fund, Date, Table type, SUM(end) as end FROM joined_balance GROUP BY Fund, Date, Table type) PIVOT (SUM(end) FOR Table type IN ('Type1', 'Type2', 'Type3')) LIMIT 10);

Observation:CREATE TABLE joined_balance_output AS (SELECT * FROM (SELECT Fund, Date, Table type, SUM(end) as end FROM joined

In [316]:
r = agent_executor.invoke({"input":"I deleted that. Can you try creating again?"})



> Entering new AgentExecutor chain...
CREATE TABLE joined_balance_output AS (SELECT Fund, Date, SUM(CASE WHEN "Table type" = 'A' THEN end ELSE 0 END) AS A, SUM(CASE WHEN "Table type" = 'B' THEN end ELSE 0 END) AS B, SUM(CASE WHEN "Table type" = 'C' THEN end ELSE 0 END) AS C FROM joined_balance GROUP BY Fund, Date);  action input
Thought: The user wants to create a table from the previous result. I need to recall the last query I used to produce the result. Then, I will create a new table named 'joined_balance_output' with the result of that query. 

Action: sql_db_query
Action Input: CREATE TABLE joined_balance_output AS (SELECT Fund, Date, SUM(CASE WHEN "Table type" = 'A' THEN end ELSE 0 END) AS A, SUM(CASE WHEN "Table type" = 'B' THEN end ELSE 0 END) AS B, SUM(CASE WHEN "Table type" = 'C' THEN end ELSE 0 END) AS C FROM joined_balance GROUP BY Fund, Date);

Observation:Error: (snowflake.connector.errors.ProgrammingError) 000904 (42000): SQL compilation error: error line 1 at positio

In [245]:
r = agent_executor.invoke({"input":"For the new values, I want total_end column's value"})
getting_df(r)



> Entering new AgentExecutor chain...
SELECT Fund, Date, 
    SUM(CASE WHEN `Table type` = 'type1' THEN total_end ELSE 0 END) AS 'type1',
    SUM(CASE WHEN `Table type` = 'type2' THEN total_end ELSE 0 END) AS 'type2',
    SUM(CASE WHEN `Table type` = 'type3' THEN total_end ELSE 0 END) AS 'type3'
FROM joined_balance
GROUP BY Fund, Date;  action input
Thought: The user wants to pivot the table such that the 'Table type' values become separate columns, with the values in these new columns being the 'total_end' column's values. I will need to construct a SQL query to achieve this. 

Action: sql_db_query_checker
Action Input: SELECT Fund, Date, 
    SUM(CASE WHEN `Table type` = 'type1' THEN total_end ELSE 0 END) AS 'type1',
    SUM(CASE WHEN `Table type` = 'type2' THEN total_end ELSE 0 END) AS 'type2',
    SUM(CASE WHEN `Table type` = 'type3' THEN total_end ELSE 0 END) AS 'type3'
FROM joined_balance
GROUP BY Fund, Date;

Observation:SELECT Fund, Date, 
    SUM(CASE WHEN `Table type` = 'ty

KeyboardInterrupt: 

In [233]:
r = agent_executor.invoke({"input":"I want total end in new columns"})



> Entering new AgentExecutor chain...
SELECT Fund, Date, SUM(CASE WHEN `Table type` = 'A' THEN end ELSE 0 END) AS 'A', SUM(CASE WHEN `Table type` = 'B' THEN end ELSE 0 END) AS 'B', SUM(CASE WHEN `Table type` = 'C' THEN end ELSE 0 END) AS 'C' FROM joined_balance GROUP BY Fund, Date;  action input
Thought: The user wants to pivot the table such that the 'Table type' values become separate columns, with the total 'end' values in these new columns. I will need to construct a SQL query to achieve this. 

Action: sql_db_query_checker
Action Input: SELECT Fund, Date, SUM(CASE WHEN `Table type` = 'A' THEN end ELSE 0 END) AS 'A', SUM(CASE WHEN `Table type` = 'B' THEN end ELSE 0 END) AS 'B', SUM(CASE WHEN `Table type` = 'C' THEN end ELSE 0 END) AS 'C' FROM joined_balance GROUP BY Fund, Date;

Observation:SELECT Fund, Date, SUM(CASE WHEN `Table type` = 'A' THEN end ELSE 0 END) AS 'A', SUM(CASE WHEN `Table type` = 'B' THEN end ELSE 0 END) AS 'B', SUM(CASE WHEN `Table type` = 'C' THEN end ELSE 0 

In [234]:
getting_df(r)

,Fund,Date,a,b,c
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,0.0,0.0,0.0
1,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,0.0,0.0,0.0
2,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,0.0,0.0,0.0
3,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,0.0,0.0,0.0
4,1745443 - LCIV REAL RETURN FND,30 Dec 22,0.0,0.0,0.0
5,1748760 - LCIV GB EQTY FND,30 Dec 22,0.0,0.0,0.0
6,1748761 - LCIV UK EQTY FND,30 Dec 22,0.0,0.0,0.0
7,1751062 - LCIV SUSTAINABLE EQTY FND,30 Dec 22,0.0,0.0,0.0
8,1762287 - LCIV GLOBAL BOND FUND,30 Dec 22,0.0,0.0,0.0
9,1762300 - LCIV MAC INC COLL FUND,30 Dec 22,0.0,0.0,0.0


In [163]:
r = agent_executor.invoke({"input":"From where column A and B come from?"})
getting_df(r)



> Entering new AgentExecutor chain...
Thought: The user is asking about the origin of columns A and B in the final result. I need to recall the past queries to find out where these columns come from. 

Final Answer: Column A and B are created when we pivoted the table. They represent the unique values from the 'Table type' column in the original 'joined_balance' table.

> Finished chain.


IndexError: list index out of range

In [ ]:
r = agent_executor.invoke({"input":"Can you remove all the column that contains start from this result?"})



> Entering new AgentExecutor chain...
joined_balance  action input
Thought: The user wants to remove all columns that contain "start" from the previous result. To do this, I need to recall the structure of the previous result and identify the columns that contain "start". Then, I can create a new SQL query that selects all columns except those containing "start". 

Action: sql_db_schema
Action Input: joined_balance

In [ ]:
getting_df(r)

In [104]:
r = agent_executor.invoke({"input":"I do not want SUM Start column from this result. Can you remove it?"})



> Entering new AgentExecutor chain...
SELECT Fund, Date, Table_type, SUM(End) FROM joined_balance GROUP BY Fund, Date, Table_type;  action input
Thought: The user wants to remove the SUM Start column from the previous result. I need to modify the previous SQL query to exclude the SUM Start column. 

Action: sql_db_query_checker
Action Input: SELECT Fund, Date, Table_type, SUM(End) FROM joined_balance GROUP BY Fund, Date, Table_type;

Observation:SELECT Fund, Date, Table_type, SUM(End) FROM joined_balance GROUP BY Fund, Date, Table_type;
SELECT Fund, Date, Table_type, SUM(End) FROM joined_balance GROUP BY Fund, Date, Table_type;  action input
The query is correct. Now I can execute it.

Action: sql_db_query
Action Input: SELECT Fund, Date, Table_type, SUM(End) FROM joined_balance GROUP BY Fund, Date, Table_type;

Observation:Error: (snowflake.connector.errors.ProgrammingError) 000904 (42000): SQL compilation error: error line 1 at position 7
invalid identifier 'FUND'
[SQL: SELECT Fund

In [106]:
getting_df(r)

,Fund,Date,Table type,"SUM(""END"")"
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Receivables,1.888321e+07
1,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Liabilities,-3.318115e+06
2,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Total net assets,8.900892e+08
3,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Liabilities,-7.503213e+05
4,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Total net assets,7.863990e+08
5,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Receivables,0.000000e+00
6,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Receivables,0.000000e+00
7,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Liabilities,-2.472860e+05
8,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Total net assets,2.150252e+08
9,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,Liabilities,-6.205894e+04


In [107]:
r = agent_executor.invoke({"input":"Can you turn 'Table type' column values to individual columns from this result?"})



> Entering new AgentExecutor chain...
SELECT DISTINCT "Table type" FROM joined_balance LIMIT 10;
  action input
Thought: The user wants to pivot the 'Table type' column into individual columns. This can be achieved using the PIVOT function in SQL. However, I need to know the unique values in the 'Table type' column to create the pivot table. I will first extract the unique values from the 'Table type' column from the joined_balance table.

Action: sql_db_query
Action Input: SELECT DISTINCT "Table type" FROM joined_balance LIMIT 10;


Observation:DF: {'Table type': ['Receivables', 'Liabilities', 'Total net assets']}
SELECT "Fund", "Date", 
MAX(CASE WHEN "Table type" = 'Receivables' THEN "End" ELSE NULL END) AS "Receivables",
MAX(CASE WHEN "Table type" = 'Liabilities' THEN "End" ELSE NULL END) AS "Liabilities",
MAX(CASE WHEN "Table type" = 'Total net assets' THEN "End" ELSE NULL END) AS "Total net assets"
FROM joined_balance
GROUP BY "Fund", "Date"
ORDER BY "Date" ASC
LIMIT 10;
  actio

In [108]:
getting_df(r)

,Fund,Date,Receivables,Liabilities,Total net assets
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,7.996314e+06,0.0,8.900892e+08
1,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,0.000000e+00,0.0,7.863990e+08
2,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,0.000000e+00,0.0,2.150252e+08
3,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,1.083894e+07,0.0,1.143294e+09
4,1745443 - LCIV REAL RETURN FND,30 Dec 22,0.000000e+00,-96.5,1.743970e+08
5,1748760 - LCIV GB EQTY FND,30 Dec 22,2.723829e+06,0.0,5.565896e+08
6,1748761 - LCIV UK EQTY FND,30 Dec 22,8.715891e+04,0.0,0.000000e+00
7,1751062 - LCIV SUSTAINABLE EQTY FND,30 Dec 22,4.258751e+07,0.0,1.238336e+09
8,1762287 - LCIV GLOBAL BOND FUND,30 Dec 22,2.537348e+08,0.0,6.509100e+08
9,1762300 - LCIV MAC INC COLL FUND,30 Dec 22,NaN,NaN,1.740521e+06


In [109]:
r = agent_executor.invoke({"input":"Let's go back to previous filter I gave you? Can you print it again?"})



> Entering new AgentExecutor chain...
SELECT Fund, Date, MAX(IF(Table_type = 'A', End, NULL)) AS 'A', MAX(IF(Table_type = 'B', End, NULL)) AS 'B', MAX(IF(Table_type = 'C', End, NULL)) AS 'C' FROM (SELECT Fund, Date, Table_type, SUM(End) as End FROM joined_balance GROUP BY Fund, Date, Table_type) GROUP BY Fund, Date;  action input
Thought: The user wants to see the result of the previous filter they provided. I need to recall the previous SQL query and execute it again. The last query was to transform 'Table type' column values to individual columns from the result of the query that aggregated start and end column values by Table type value and grouped them by Fund, Date and Table type columns, but without the SUM Start column.

Action: sql_db_query
Action Input: SELECT Fund, Date, MAX(IF(Table_type = 'A', End, NULL)) AS 'A', MAX(IF(Table_type = 'B', End, NULL)) AS 'B', MAX(IF(Table_type = 'C', End, NULL)) AS 'C' FROM (SELECT Fund, Date, Table_type, SUM(End) as End FROM joined_balance

In [110]:
getting_df(r)

,Fund,Date,a,b,c
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,None,None,None
1,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,None,None,None
2,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,None,None,None
3,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,None,None,None
4,1745443 - LCIV REAL RETURN FND,30 Dec 22,None,None,None
5,1748760 - LCIV GB EQTY FND,30 Dec 22,None,None,None
6,1748761 - LCIV UK EQTY FND,30 Dec 22,None,None,None
7,1750023 - LCIV GB EQ FOCUS FND,30 Dec 22,None,None,None
8,1751062 - LCIV SUSTAINABLE EQTY FND,30 Dec 22,None,None,None
9,1751063 - LCIV EQTY INCOME FND,30 Dec 22,None,None,None


In [109]:
r = agent_executor.invoke({"input":"Can you turn table types from this to columns ?"})



> Entering new AgentExecutor chain...
SELECT DISTINCT "Table type" FROM joined_balance;  action input
Thought: The user wants to pivot the table, turning the unique values in the 'Table type' column into separate columns. This is a common operation in data analysis, but it's not directly supported in SQL. However, it can be achieved using a combination of aggregate functions and CASE statements. I need to know the unique values in the 'Table type' column to construct the query. 

Action: sql_db_query
Action Input: SELECT DISTINCT "Table type" FROM joined_balance;

Observation:DF: {'Table type': ['Receivables', 'Liabilities', 'Total net assets']}
SELECT "Fund", "Date", 
SUM(CASE WHEN "Table type" = 'Receivables' THEN "start" + "end" ELSE 0 END) AS "Receivables", 
SUM(CASE WHEN "Table type" = 'Liabilities' THEN "start" + "end" ELSE 0 END) AS "Liabilities", 
SUM(CASE WHEN "Table type" = 'Total net assets' THEN "start" + "end" ELSE 0 END) AS "Total net assets" 
FROM joined_balance 
GROUP

In [115]:
r = agent_executor.invoke({"input":"Can you produce this table to ouput table?"})



> Entering new AgentExecutor chain...
CREATE TABLE output_test_2 AS SELECT * FROM joined_balance;  action input
Thought: I need to create a SQL query to produce the output table as per the user's request. 

Action: sql_db_query_checker
Action Input: CREATE TABLE output_test_2 AS SELECT * FROM joined_balance;

Observation:CREATE TABLE output_test_2 AS SELECT * FROM joined_balance;
CREATE TABLE output_test_2 AS SELECT * FROM joined_balance;  action input
The query has been checked and it is correct. Now, I can execute it to create the output table.

Action: sql_db_query
Action Input: CREATE TABLE output_test_2 AS SELECT * FROM joined_balance;

Observation:Error: (snowflake.connector.errors.ProgrammingError) 002002 (42710): SQL compilation error:
Object 'OUTPUT_TEST_2' already exists.
[SQL: CREATE TABLE output_test_2 AS SELECT * FROM joined_balance;]
(Background on this error at: https://sqlalche.me/e/14/f405)
The query execution failed because a table with the name 'output_test_2' alre

In [113]:
r = agent_executor.invoke({"input":"output_test_2"})



> Entering new AgentExecutor chain...
CREATE TABLE output_test_2 AS SELECT * FROM joined_balance;  action input
Thought: The user wants to create an output table from the previously manipulated data. I need to create a SQL query to create this table and name it as per the user's request. 

Action: sql_db_query_checker
Action Input: CREATE TABLE output_test_2 AS SELECT * FROM joined_balance;

Observation:CREATE TABLE output_test_2 AS SELECT * FROM joined_balance;
CREATE TABLE output_test_2 AS SELECT * FROM joined_balance;  action input
The query syntax is correct. Now I can execute the query to create the output table.

Action: sql_db_query
Action Input: CREATE TABLE output_test_2 AS SELECT * FROM joined_balance;

Observation:DF: {'status': ['Table OUTPUT_TEST_2 successfully created.']}
The output table has been successfully created with the name "output_test_2".

Final Answer: The output table has been successfully created. <output_name>output_test_2</output_name>

> Finished chain.


In [ ]:
r = agent_executor.invoke({"input":"Can you turn table types from this to column names?"})



> Entering new AgentExecutor chain...
Thought: I need additional information on that question
Final Answer: Could you please specify which table you are referring to and which column should be turned into column names?

> Finished chain.


In [24]:
df = getting_df(r)
df

,Fund,Date,Table type,total_start,total_end
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Receivables,2.208791e+07,1.888321e+07
1,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Liabilities,-8.797284e+06,-3.318115e+06
2,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Total net assets,1.229889e+09,8.900892e+08
3,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Liabilities,-5.937032e+06,-7.503213e+05
4,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Total net assets,7.727695e+08,7.863990e+08
5,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Receivables,0.000000e+00,0.000000e+00
6,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Receivables,2.118886e+06,0.000000e+00
7,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Liabilities,-8.974092e+05,-2.472860e+05
8,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Total net assets,2.117766e+08,2.150252e+08
9,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,Liabilities,-3.629060e+05,-6.205894e+04


In [111]:
dataframe

,Fund,Date,Receivables,Liabilities,Total net assets
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,4.097112e+07,-1.211540e+07,2.119978e+09
1,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,0.000000e+00,-6.687353e+06,1.559169e+09
2,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,2.118886e+06,-1.144695e+06,4.268018e+08
3,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,1.083894e+07,-4.249650e+05,2.290189e+09
4,1745443 - LCIV REAL RETURN FND,30 Dec 22,2.075014e+06,-4.763489e+04,3.429333e+08
5,1748760 - LCIV GB EQTY FND,30 Dec 22,1.114676e+07,-1.181084e+07,1.112301e+09
6,1748761 - LCIV UK EQTY FND,30 Dec 22,1.741750e+05,-1.536122e+05,4.138850e+03
7,1751062 - LCIV SUSTAINABLE EQTY FND,30 Dec 22,4.996496e+07,-1.110161e+07,2.496362e+09
8,1762287 - LCIV GLOBAL BOND FUND,30 Dec 22,5.245453e+08,-2.239304e+08,1.204780e+09
9,1762300 - LCIV MAC INC COLL FUND,30 Dec 22,0.000000e+00,0.000000e+00,2.418374e+06


In [ ]:
"Can you give me output table from joined_balance and cash tables? a"

In [101]:
r = agent_executor.invoke({"input":"balance"})



> Entering new AgentExecutor chain...
Thought: The user has only mentioned "balance" which is a table name in our database. However, the user has not specified what they want to do with the balance table. I need more information to proceed.

Final Answer: Could you please specify what you would like to do with the "balance" table?

> Finished chain.


In [90]:
last_observation = r["intermediate_steps"][-1][1]

In [57]:
import ast 
if last_observation.startswith("DF: "):
    last_observation_data = ast.literal_eval(last_observation.split("DF: ")[1])
    print(last_observation_data)

{'Fund Name': ['1736917 - LCIV GB ALPHA GROWTH FND', '1736938 - LCIV DIVERSIFD GROWTH FND', '1740040 - LCIV GB TOTAL RETURN FND', '1740041 - LCIV ABSOLUTE RETURN FND', '1745443 - LCIV REAL RETURN FND', '1748760 - LCIV GB EQTY FND', '1748761 - LCIV UK EQTY FND', '1750023 - LCIV GB EQ FOCUS FND', '1751062 - LCIV SUSTAINABLE EQTY FND', '1751063 - LCIV EQTY INCOME FND'], 'Receivables': ['18883208.64', '0.0', '0.0', '10838935.29', '0.0', '7153697.63', '87428.49', '7942947.05', '45899478.54', '573613.5'], 'Liabilities': ['-14815065.4', '-791115.69', '-287602.49', '-110696.16', '-32124.45', '-7761477.01', '-245046.85', '-5333629.91', '-90197952.07', '-1520495.49'], 'Total Net Assets': ['2730435768.31', '694714006.32', '244329466.82', '1116910867.09', '180707428.16', '786682155.78', '810.5', '964393596.34', '1246057757.18', '42791.0'], 'Date': ['30 Sep 21', '30 Sep 21', '30 Sep 21', '30 Sep 21', '30 Sep 21', '30 Sep 21', '30 Sep 21', '30 Sep 21', '30 Sep 21', '30 Sep 21']}


In [ ]:
if output_table_name:


In [103]:
r

{'input': 'balance',
 'history': [HumanMessage(content='Hellow'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='Can you get 10 rows from balance table?'),
  AIMessage(content='Here it is.'),
  HumanMessage(content='What are available tables?'),
  AIMessage(content='The available tables in the database are balance, cash, investment, and turnover.\n')],
 'output': 'Could you please specify what you would like to do with the "balance" table?',
 'output_table_name': None,
 'options_text': [],
 'intermediate_steps': []}

In [245]:
r = agent_executor.invoke({"input":"Can you give me the first 10 rows from balance sheet?"})

{'input': 'Can you give me the first 10 rows from balance sheet?',
 'history': [],
 'output': 'I\'m sorry, but the table "balance sheet" does not exist in the database. The available tables are balance, cash, investment, and turnover. Could you please specify the correct table name?\n',
 'output_table_name': None,
 'options_text': ['balance', 'cash', 'investment', 'turnover'],
 'intermediate_steps': [(AgentAction(tool='sql_db_list_tables', tool_input='', log='Thought: To get the first 10 rows from the balance sheet, I need to write a SQL query. However, before that, I should confirm if the table "balance sheet" exists in the database.\nAction: sql_db_list_tables\nAction Input: "" '),
   'balance, cash, investment, turnover')]}

In [205]:
# from langchain.memory.chat_message_histories import RedisChatMessageHistory
# message_history = RedisChatMessageHistory(
#     url="redis://localhost:6379/0", ttl=600, session_id="my-session"
# )


In [11]:
from langchain.memory.chat_message_histories import StreamlitChatMessageHistory

In [17]:
import streamlit as st
st.session_state["langchain"] = None

In [19]:
st.session_state.key = None

In [30]:
st.session_state

# creating streamlit chat message history

In [174]:
# message_history = StreamlitChatMessageHistory(key="langchain")

In [ ]:
message_history

In [187]:
memory

ConversationBufferWindowMemory(output_key='output', input_key='input', return_messages=True, k=10)



> Entering new AgentExecutor chain...
Thought: The user wants to see the first 10 rows from the balance table. I need to construct a SQL query to fetch this data.

Action: sql_db_query
Action Input: SELECT * FROM balance LIMIT 10  llm output
SELECT * FROM balance LIMIT 10  action input
Thought: The user wants to see the first 10 rows from the balance table. I need to construct a SQL query to fetch this data.

Action: sql_db_query
Action Input: SELECT * FROM balance LIMIT 10

Observation:{'Fund Name': ['1736917 - LCIV GB ALPHA GROWTH FND', '1736938 - LCIV DIVERSIFD GROWTH FND', '1740040 - LCIV GB TOTAL RETURN FND', '1740041 - LCIV ABSOLUTE RETURN FND', '1745443 - LCIV REAL RETURN FND', '1748760 - LCIV GB EQTY FND', '1748761 - LCIV UK EQTY FND', '1750023 - LCIV GB EQ FOCUS FND', '1751062 - LCIV SUSTAINABLE EQTY FND', '1751063 - LCIV EQTY INCOME FND'], 'Receivables': ['18883208.64', '0.0', '0.0', '10838935.29', '0.0', '7153697.63', '87428.49', '7942947.05', '45899478.54', '573613.5'], '

In [179]:
r = agent_executor.invoke({"input":"balance"})



> Entering new AgentExecutor chain...
SELECT * FROM balance LIMIT 10  action input
Thought: The user has specified the "balance" table. I will now construct a SQL query to fetch the first 10 rows from this table.
Action: sql_db_query_checker
Action Input: SELECT * FROM balance LIMIT 10

Observation:SELECT * FROM balance LIMIT 10
SELECT * FROM balance LIMIT 10  action input
The query is correct. Now I will execute it to fetch the data from the "balance" table.
Action: sql_db_query
Action Input: SELECT * FROM balance LIMIT 10

Observation:{'Fund Name': ['1736917 - LCIV GB ALPHA GROWTH FND', '1736938 - LCIV DIVERSIFD GROWTH FND', '1740040 - LCIV GB TOTAL RETURN FND', '1740041 - LCIV ABSOLUTE RETURN FND', '1745443 - LCIV REAL RETURN FND', '1748760 - LCIV GB EQTY FND', '1748761 - LCIV UK EQTY FND', '1750023 - LCIV GB EQ FOCUS FND', '1751062 - LCIV SUSTAINABLE EQTY FND', '1751063 - LCIV EQTY INCOME FND'], 'Receivables': ['18883208.64', '0.0', '0.0', '10838935.29', '0.0', '7153697.63', '874

In [183]:
r

{'input': 'balance',
 'history': [HumanMessage(content='Can you give me the first 10 rows from balance sheet?'),
  AIMessage(content='I\'m sorry, but the table "balance_sheet" does not exist in the database. The available tables are "balance", "cash", "investment", and "turnover". Could you please specify the correct table from which you want to fetch the data?\n<list>balance,cash,investment,turnover</list>')],
 'output': 'Here are the first 10 rows from the "balance" table:\n\n1. Fund Name: 1736917 - LCIV GB ALPHA GROWTH FND, Receivables: 18883208.64, Liabilities: -14815065.4, Total Net Assets: 2730435768.31, Date: 30 Sep 21\n2. Fund Name: 1736938 - LCIV DIVERSIFD GROWTH FND, Receivables: 0.0, Liabilities: -791115.69, Total Net Assets: 694714006.32, Date: 30 Sep 21\n3. Fund Name: 1740040 - LCIV GB TOTAL RETURN FND, Receivables: 0.0, Liabilities: -287602.49, Total Net Assets: 244329466.82, Date: 30 Sep 21\n4. Fund Name: 1740041 - LCIV ABSOLUTE RETURN FND, Receivables: 10838935.29, Liab

In [84]:
r = agent_executor.invoke({"input":"Output"})



> Entering new AgentExecutor chain...
balance, cash  action input
Thought: The user has provided the name for the output table. Now, I need to check the schema of the tables 'balance' and 'cash' to understand their structure and to ensure they exist in the database.
Action: sql_db_schema
Action Input: balance, cash

Observation:
CREATE TABLE balance (
	"Fund Name" VARCHAR(200), 
	"Receivables" FLOAT, 
	"Liabilities" FLOAT, 
	"Total Net Assets" FLOAT, 
	"Date" VARCHAR(200)
)

/*
3 rows from balance table:
Fund Name	Receivables	Liabilities	Total Net Assets	Date
1736917 - LCIV GB ALPHA GROWTH FND	18883208.64	-14815065.4	2730435768.31	30 Sep 21
1736938 - LCIV DIVERSIFD GROWTH FND	0.0	-791115.69	694714006.32	30 Sep 21
1740040 - LCIV GB TOTAL RETURN FND	0.0	-287602.49	244329466.82	30 Sep 21
*/


CREATE TABLE cash (
	"Valuation Date" DATE, 
	"Account ID (FA)" VARCHAR(50), 
	"Account ID (FA)/Name" VARCHAR(100), 
	"Account ID (Custody)" VARCHAR(50), 
	"Security Description (Short)" VARCHAR(25

In [85]:
r["output"]

"The output table 'Output' has been successfully created from the 'balance' and 'cash' tables. <output_name>Output</output_name>"

In [55]:
memory.chat_memory.messages[1].type

'ai'

In [22]:
r["output"]

'I\'m sorry, but the table "balance sheet" does not exist in the database. The available tables are: balance, cash, investment, lciv_output, movies, output, outputs, outputtest, turnover. Could you please specify the correct table name?\n<list>balance,cash,investment,lciv_output,movies,output,outputs,outputtest,turnover</list>'

In [338]:
agent_executor.invoke({"input":"Yes, balance is the table I want"})



> Entering new AgentExecutor chain...
SELECT * FROM balance LIMIT 10
  action input
Thought: The user wants to see the first 10 rows from the 'balance' table. I need to construct a SQL query to retrieve this data.

Action: sql_db_query_checker

Action Input: SELECT * FROM balance LIMIT 10


Observation:SELECT * FROM balance LIMIT 10
SELECT * FROM balance LIMIT 10
  action input
The query is correct. Now I can execute it to retrieve the data from the 'balance' table.

Action: sql_db_query

Action Input: SELECT * FROM balance LIMIT 10


Observation:[('1736917 - LCIV GB ALPHA GROWTH FND', 18883208.64, -14815065.4, 2730435768.31, '30 Sep 21'), ('1736938 - LCIV DIVERSIFD GROWTH FND', 0.0, -791115.69, 694714006.32, '30 Sep 21'), ('1740040 - LCIV GB TOTAL RETURN FND', 0.0, -287602.49, 244329466.82, '30 Sep 21'), ('1740041 - LCIV ABSOLUTE RETURN FND', 10838935.29, -110696.16, 1116910867.09, '30 Sep 21'), ('1745443 - LCIV REAL RETURN FND', 0.0, -32124.45, 180707428.16, '30 Sep 21'), ('1748760

{'input': 'Yes, balance is the table I want',
 'history': [HumanMessage(content='Can you show me the first 10 rows from balance sheet?'),
  AIMessage(content="The 'balance sheet' table does not exist in the database. Could you please specify the correct table name? The available tables are 'balance', 'cash', 'investment', and 'turnover'.")],
 'output': "Here are the first 10 rows from the 'balance' table:\n\n1. '1736917 - LCIV GB ALPHA GROWTH FND', 18883208.64, -14815065.4, 2730435768.31, '30 Sep 21'\n2. '1736938 - LCIV DIVERSIFD GROWTH FND', 0.0, -791115.69, 694714006.32, '30 Sep 21'\n3. '1740040 - LCIV GB TOTAL RETURN FND', 0.0, -287602.49, 244329466.82, '30 Sep 21'\n4. '1740041 - LCIV ABSOLUTE RETURN FND', 10838935.29, -110696.16, 1116910867.09, '30 Sep 21'\n5. '1745443 - LCIV REAL RETURN FND', 0.0, -32124.45, 180707428.16, '30 Sep 21'\n6. '1748760 - LCIV GB EQTY FND', 7153697.63, -7761477.01, 786682155.78, '30 Sep 21'\n7. '1748761 - LCIV UK EQTY FND', 87428.49, -245046.85, 810.5, '

In [339]:
r = agent_executor.invoke({"input":"Can you give me output from balance and cash table? Make sure that receivables is more than 0"})



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to create a SQL query that combines the 'balance' and 'cash' tables and filters for rows where 'receivables' is more than 0. However, I need to know the name of the output table from the user before I can proceed.

Final Answer: Could you please provide a name for the output table?

> Finished chain.


In [340]:
r = agent_executor.invoke({"input":"Name it outputtest"})



> Entering new AgentExecutor chain...
balance, cash  action input
Thought: I need to create a new table named 'outputtest' that contains all the columns from both 'balance' and 'cash' tables. Since the columns in the tables are different, I will need to fill in the missing columns with NULL values to make the number of columns equal. I will also need to filter the rows where 'receivables' is more than 0. First, I need to check the schema of both tables to know the columns.

Action: sql_db_schema
Action Input: balance, cash

Observation:
CREATE TABLE balance (
	"Fund Name" VARCHAR(200), 
	"Receivables" FLOAT, 
	"Liabilities" FLOAT, 
	"Total Net Assets" FLOAT, 
	"Date" VARCHAR(200)
)

/*
3 rows from balance table:
Fund Name	Receivables	Liabilities	Total Net Assets	Date
1736917 - LCIV GB ALPHA GROWTH FND	18883208.64	-14815065.4	2730435768.31	30 Sep 21
1736938 - LCIV DIVERSIFD GROWTH FND	0.0	-791115.69	694714006.32	30 Sep 21
1740040 - LCIV GB TOTAL RETURN FND	0.0	-287602.49	244329466.82	

In [342]:
r["output"]

"Here are the first 10 rows from the 'outputtest' table:\n\n1. '1736917 - LCIV GB ALPHA GROWTH FND', 18883208.64, -14815065.4, 2730435768.31, '30 Sep 21'\n2. '1740041 - LCIV ABSOLUTE RETURN FND', 10838935.29, -110696.16, 1116910867.09, '30 Sep 21'\n3. '1748760 - LCIV GB EQTY FND', 7153697.63, -7761477.01, 786682155.78, '30 Sep 21'\n4. '1748761 - LCIV UK EQTY FND', 87428.49, -245046.85, 810.5, '30 Sep 21'\n5. '1750023 - LCIV GB EQ FOCUS FND', 7942947.05, -5333629.91, 964393596.34, '30 Sep 21'\n6. '1751062 - LCIV SUSTAINABLE EQTY FND', 45899478.54, -90197952.07, 1246057757.18, '30 Sep 21'\n7. '1751063 - LCIV EQTY INCOME FND', 573613.5, -1520495.49, 42791.0, '30 Sep 21'\n8. '1762287 - LCIV GLOBAL BOND FUND', 263242527.88, -281412087.59, 495594422.05, '30 Sep 21'\n9. '1777123 - LCIV GLBAL EQUITY CORE FD', 985048.58, -1533762.86, 551872128.55, '30 Sep 21'\n10. '1778174 - LCIV EM MARKET EQ FND', 171855476.6, -170417095.36, 581977564.61, '30 Sep 21'"

In [319]:
r = agent_executor.invoke({"input":"You can replace it"})

"The 'outputtest' table has been successfully created."

In [344]:
# points

# make it more user friendly
# make it produce output with selections
# create downloadable excel file after creating table